# **From Dictionary-Based Translation to Attention Mechanisms**

This notebook walks through the progression from a **basic dictionary lookup translation system** to a **parallelized attention-based approach**, inspired by Transformer models.

- **Basic word translation** using a hardcoded dictionary.
- **Tokenization** of text into words.
- **Error handling** with fuzzy matching (Levenshtein distance) for misspelled or missing words.
- **Vocabulary creation** and **one-hot encoding**.
- Representing a translation dictionary with **matrix multiplication**.
- **Decoding** one-hot vectors back into words.
- Introduction to the **softmax-based attention mechanism**:
  - Scaled dot-product attention.
  - Transition from sequential to **parallelized translation**.
- How these steps mirror components in **modern neural machine translation** systems.

1. A naive dictionary translator.
2. A matrix-based translator.
3. An attention-based translator that processes entire sentences in parallel.


### Importing required libraries


In [4]:
import torch
from Levenshtein import distance

# suppress warnings
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

## Translation

- A `dictionary` is defined, mapping French words to their English equivalents, forming the basis of translation logic.


In [6]:
dictionary = {
    'le': 'the',
    'chat': 'cat',
    'est': 'is',
    'sous': 'under',
    'la': 'the',
    'table': 'table'
}

- The `tokenize` function is responsible for breaking down a sentence into individual words.
- The `translate` function uses this `tokenize` function to split the input sentence and then translates each word according to the dictionary. The translated words are concatenated to form the output sentence.


In [7]:
def tokenize(text):
    return text.split()


def translate(sentence):
    out = ''
    for token in tokenize(sentence):
        out += dictionary[token] + ' '
    return out.strip()

In [8]:
translate("le chat est sous la table")

'the cat is under the table'

### Improvement: What if the 'key' is not in the dictionary?

- **find_closest_key Function**: This function find the closest key in the dictionary to a given query word. It uses the **Levenshtein distance** (a measure of the difference between two sequences) to find the dictionary key with the minimum distance to the query, suggesting a similar word if an exact match isn't found.

A simple form of error handling and fuzzy matching in translation systems, allowing for more flexible and fault-tolerant translations.


In [9]:
def find_closest_key(query):
    """
    The function computes the Levenshtein distance between the query and each key in the dictionary.
    The Levenshtein distance is a measure of the number of single-character edits required to change one word into the other.
    """
    closest_key, min_dist = None, float('inf')
    for key in dictionary.keys():
        dist = distance(query, key)
        if dist < min_dist:
            min_dist, closest_key = dist, key
    return closest_key


def translate(sentence):
    """
    This function tokenizes the input sentence into words and finds the closest translation for each word.
    It constructs the translated sentence by appending the translated words together.
    """
    out = ''
    for query in tokenize(sentence): 
        key = find_closest_key(query) 
        out += dictionary[key] + ' '  
    return out.strip()

In [10]:
translate("tables")

'table'

### Define vocabularies


In [11]:
# Create and sort the input vocabulary from the dictionary's keys
vocabulary_in = sorted(list(set(dictionary.keys())))
print(f"Vocabulary input ({len(vocabulary_in)}): {vocabulary_in}")

# Create and sort the output vocabulary from the dictionary's values
vocabulary_out = sorted(list(set(dictionary.values())))
print(f"Vocabulary output ({len(vocabulary_out)}): {vocabulary_out}")

Vocabulary input (6): ['chat', 'est', 'la', 'le', 'sous', 'table']
Vocabulary output (5): ['cat', 'is', 'table', 'the', 'under']


### Encode tokens using 'one hot' encoding


In [12]:
# Function to convert a list of vocabulary words into one-hot encoded vectors
def encode_one_hot(vocabulary):
    one_hot = dict()                  # Initialize a dictionary to hold one-hot encodings
    LEN = len(vocabulary)             # The length of each one-hot encoded vector will be equal to the vocabulary size
    
    for i, key in enumerate(vocabulary):
        one_hot_vector = torch.zeros(LEN)
        one_hot_vector[i] = 1
        one_hot[key] = one_hot_vector
        print(f"{key}\t: {one_hot[key]}")
    
    return one_hot

In [13]:
one_hot_in = encode_one_hot(vocabulary_in)
print('=' * 20)
one_hot_out = encode_one_hot(vocabulary_out)

chat	: tensor([1., 0., 0., 0., 0., 0.])
est	: tensor([0., 1., 0., 0., 0., 0.])
la	: tensor([0., 0., 1., 0., 0., 0.])
le	: tensor([0., 0., 0., 1., 0., 0.])
sous	: tensor([0., 0., 0., 0., 1., 0.])
table	: tensor([0., 0., 0., 0., 0., 1.])
cat	: tensor([1., 0., 0., 0., 0.])
is	: tensor([0., 1., 0., 0., 0.])
table	: tensor([0., 0., 1., 0., 0.])
the	: tensor([0., 0., 0., 1., 0.])
under	: tensor([0., 0., 0., 0., 1.])


### Creating a 'dictionary' using matrix multiplication

A representation of dictionary suitable for neural network operations:

- **Matrix creation**: Using PyTorch's `torch.stack`, convert the one-hot encoded vectors for both input (`K`) and output (`V`) vocabularies into tensors. `K` is constructed from the input vocabulary's one-hot vectors, and `V` from the output vocabulary's vectors. These tensors can be thought of as a look-up table that our model will use to associate input tokens with output tokens.

- Each row in `K` corresponds to a word in the input language represented as a one-hot vector, and each row in `V` corresponds to the respective translated word in the output language.

- **Query example**: An example shows how to use matrix operations to find a translation. Look up the one-hot vector for the word "sous" from the input vocabulary (`q`). Then demonstrate how to find its corresponding translation by performing matrix multiplication with the transpose of `K` (i.e., `q @ K.T`) to identify the index and then use that index to select the relevant row from `V`. This process mimics the lookup.

In [14]:
K = torch.stack([one_hot_in[k] for k in dictionary.keys()])
print(K)

tensor([[0., 0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1.]])


In [15]:
V = torch.stack([one_hot_out[k] for k in dictionary.values()])
print(V)

tensor([[0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0.]])


look up a translation for a given word using matrix operations
we take the one-hot representation of 'sous' from the input vocabulary

In [16]:
q = one_hot_in['sous']
print("Query token :", q)

Query token : tensor([0., 0., 0., 0., 1., 0.])


Select the corresponding key vector in K (input dictionary matrix) using matrix multiplication. This operation gives us the index where 'sous' would be '1' in the one-hot encoded input matrix

In [18]:
print("Select key (K) :", q @ K.T)

Select key (K) : tensor([0., 0., 0., 1., 0., 0.])


Use the index found from the key selection to find the corresponding value vector in V (output dictionary matrix). This operation selects the row from V that is the translation of 'sous' in the output vocabulary

In [19]:
print("Select value (V):", q @ K.T @ V)

Select value (V): tensor([0., 0., 0., 0., 1.])


### Decode one-hot vector
The `decode_one_hot` function is used to decode a one-hot encoded vector back into the corresponding token (word). It does this by finding the token whose one-hot representation has the highest cosine similarity with the given vector, which is effectively just the dot product due to the nature of one-hot vectors.


In [20]:
def decode_one_hot(one_hot, vector):
    best_key, best_cosine_sim = None, 0
    for k, v in one_hot.items():
        cosine_sim = torch.dot(vector, v)
        if cosine_sim > best_cosine_sim:
            best_cosine_sim, best_key = cosine_sim, k
    return best_key

The operation $q \cdot K^T \cdot V$ allows us to build a dictionary-like structure from a set of vectors

In [21]:
for k, v in one_hot_in.items():
    print(k, " = ", decode_one_hot(one_hot_out, one_hot_in[k] @ K.T @ V))

chat  =  cat
est  =  is
la  =  the
le  =  the
sous  =  under
table  =  table


### Matrix-based translate function
The `translate` function now leverages matrix operations to perform the translation. For each token in the input sentence, it finds its one-hot vector, multiplies it with the matrices `K.T` and `V` to find the corresponding one-hot vector in the output vocabulary, and then decodes this vector to get the translated word.


In [22]:
def translate(sentence):
    sentence_out = ''
    for token_in in tokenize(sentence):
        q = one_hot_in[token_in]                      # Find the one-hot vector for the token
        out = q @ K.T @ V                             # Multiply with the input and output matrices to find the translation
        token_out = decode_one_hot(one_hot_out, out)  # Decode the output one-hot vector to a token
        sentence_out += token_out + ' '               # Append the translated token to the output sentence
    return sentence_out.strip()

In [23]:
translate("le chat est sous la table")

'the cat is under the table'

**This enhanced approach shows how neural network models can translate languages by representing the translation dictionary as matrices and using vector operations.**


**Lets go towards the implementation of "Attention" in neural networks:**


### Softmax function for similarity
Similar tokens will have similar vectors, and a softmax function is applied to the output of the matrix multiplication of the query vector `q` and the transpose of the matrix `K`. The softmax function converts these values into probabilities, emphasizing the most similar token while still considering the others.


In [24]:
print('E_{table} = ', one_hot_in['table'])

E_{table} =  tensor([0., 0., 0., 0., 0., 1.])


New equation is:
$$
softmax(q \cdot K^T) \cdot V
$$

Let's adjust by the dimensionality of the query vector

$$
softmax\left( \frac{q \cdot K^T}{\sqrt{d}} \right) \cdot V
$$  



Scaled Dot-Product Attention in Transformers - and it’s there to keep the numbers stable during training.

    Q = query matrix
    K = key matrix
    V = value matrix
    d = dimension of each attention head (head_size)


### Translation with attention mechanism
- The progression from simple look-up-based translation to an attention-based approach, a key component of modern neural translation models.

The `translate` function is modified to use the softmax function as a way of applying attention. It first finds the one-hot vector for the token, then applies the softmax function to the dot product of `q` and `K.T`, scales it by the square root of the dimensionality (for normalization purposes), and finally multiplies this by `V` to get the output vector.  

    The softmax function is used to calculate a weighted sum of the V vectors, focusing on the most relevant vector for translation.
    The result is a blend of value vectors, with the most relevant ones contributing more to the output.

In [25]:
def translate(sentence):
    sentence_out = '' 
    for token_in in tokenize(sentence):
        q = one_hot_in[token_in]              # Get the one-hot vector for the current token
        # Apply softmax to the scaled dot product of q and K.T, then multiply by V
        # This selects the most relevant translation vector from V
        out = torch.softmax(q @ K.T, dim=0) @ V
        token_out = decode_one_hot(one_hot_out, out)  # Decode the output vector to a token
        sentence_out += token_out + ' '  
    return sentence_out.strip()

translate("le chat est sous la table")

'the cat is under the table'

the attention mechanism implemented using softmax works as intended.


**Improvement in the translation process by handling all queries in parallel:**


### Creating the 'Q' matrix
The matrix `Q` is constructed by stacking the one-hot encoded vectors of all tokens in the input sentence. This parallelizes the process of preparing the query vectors, which is more efficient than doing it sequentially.

In [26]:
# The sentence we want to translate
sentence = "le chat est sous la table"

# Stack all the one-hot encoded vectors for the tokens in the sentence to form the Q matrix
Q = torch.stack([one_hot_in[token] for token in tokenize(sentence)])

print(Q)

tensor([[0., 0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1.]])


$$
Attention(Q, K, V) = softmax\left( \frac{Q \cdot K^T}{\sqrt{d}} \right) V
$$


In [27]:
Q @ K.T

tensor([[1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1.]])

### Updated translate function
The `translate` function is revised to use matrix multiplication across the entire sentence. Instead of translating word by word, it now uses the "Q" matrix to perform the operation in parallel for all words.

In [28]:
def translate(sentence):
    """
    Translate a sentence using matrix multiplication in parallel.
    This function replaces the iterative approach with a single matrix multiplication step,
    applying the attention mechanism across all tokens at once.
    """
    # Tokenize the sentence and stack the one-hot vectors to form the Q matrix
    Q = torch.stack([one_hot_in[token] for token in tokenize(sentence)])
    
    # Apply softmax to the dot product of Q and K.T and multiply by V
    # This will give us the output vectors for all tokens in parallel
    out = torch.softmax(Q @ K.T, 0) @ V
    
    # Decode each one-hot vector in the output to the corresponding token
    # And join the tokens to form the translated sentence
    return ' '.join([decode_one_hot(one_hot_out, o) for o in out])
    
# Test the function to ensure it produces the correct translation
translate("le chat est sous la table")

'the cat is under the table'

- **Efficiency improvement**: By applying operations to the entire sentence at once, this approach simulates a key aspect of the actual attention mechanism used in neural networks, which is processing multiple components of input data in parallel for faster computation.

- **Test output**: The updated function correctly translates the French sentence "le chat est sous la table" to "the cat is under the table", confirming that the parallelization works effectively.
